In [1]:
# 1️⃣ Install dependencies
!pip install streamlit colabcode pyngrok scikit-learn joblib pandas numpy

Requested uvicorn==0.13.1 from https://files.pythonhosted.org/packages/ef/67/546c35e9fffb585ea0608ba3bdcafe17ae402e304367203d0b08d6c23051/uvicorn-0.13.1-py3-none-any.whl (from colabcode) has invalid metadata: .* suffix can only be used with `==` or `!=` operators
    python-dotenv (>=0.13.*) ; extra == 'standard'
                   ~~~~~~~^
Please use pip<24.1 if you need to use this version.
INFO: pip is looking at multiple versions of colabcode to determine which version is compatible with other requirements. This could take a while.
  Using cached uvicorn-0.13.1-py3-none-any.whl.metadata (4.6 kB)
Requested uvicorn==0.13.1 from https://files.pythonhosted.org/packages/ef/67/546c35e9fffb585ea0608ba3bdcafe17ae402e304367203d0b08d6c23051/uvicorn-0.13.1-py3-none-any.whl (from colabcode) has invalid metadata: .* suffix can only be used with `==` or `!=` operators
    python-dotenv (>=0.13.*) ; extra == 'standard'
                   ~~~~~~~^
Please use pip<24.1 if you need to use this versio

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
import joblib
import ipywidgets as widgets
from IPython.display import display, clear_output

In [3]:
#file upload
from google.colab import files
uploaded = files.upload()

Saving diabetes.csv to diabetes.csv


In [4]:
print("Dataset Loaded Successfully!")
data = pd.read_csv('diabetes.csv')
print(data.head())

Dataset Loaded Successfully!
   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0          NaN      148             72             35        0  33.6   
1          1.0       85             66             29        0  26.6   
2          8.0      183             64              0        0  23.3   
3          1.0       89             66             23       94  28.1   
4          0.0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  


In [5]:
# Handle zero values
cols_with_zero = ['Glucose','BloodPressure','SkinThickness','Insulin','BMI']
for col in cols_with_zero:
    data[col] = data[col].replace(0, data[col].median())

# Handle NaN values in 'Pregnancies'
data['Pregnancies'].fillna(data['Pregnancies'].median(), inplace=True)


X = data.drop('Outcome', axis=1)
y = data['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

log_model = LogisticRegression()
log_model.fit(X_train_scaled, y_train)

tree_model = DecisionTreeClassifier(random_state=42)
tree_model.fit(X_train, y_train)

# Save models and scaler
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(log_model, 'log_model.pkl')
joblib.dump(tree_model, 'tree_model.pkl')

/tmp/ipython-input-299665014.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Pregnancies'].fillna(data['Pregnancies'].median(), inplace=True)


['tree_model.pkl']

In [6]:
# 2️⃣ Define Interactive Widgets
Pregnancies = widgets.FloatText(description='Pregnancies', value=0)
Glucose = widgets.FloatText(description='Glucose', value=120)
BloodPressure = widgets.FloatText(description='BloodPressure', value=70)
SkinThickness = widgets.FloatText(description='SkinThickness', value=20)
Insulin = widgets.FloatText(description='Insulin', value=79)
BMI = widgets.FloatText(description='BMI', value=25)
DiabetesPedigreeFunction = widgets.FloatText(description='DPF', value=0.5)
Age = widgets.FloatText(description='Age', value=30)

button = widgets.Button(description='Predict')
output = widgets.Output()

In [7]:
# 3️⃣ Define Prediction Function
def on_button_clicked(b):
    with output:
        clear_output()
        # Prepare input
        patient_data = np.array([[Pregnancies.value, Glucose.value, BloodPressure.value,
                                  SkinThickness.value, Insulin.value, BMI.value,
                                  DiabetesPedigreeFunction.value, Age.value]])

        # Load models
        scaler = joblib.load('scaler.pkl')
        log_model = joblib.load('log_model.pkl')
        tree_model = joblib.load('tree_model.pkl')

        # Scale for logistic regression
        patient_scaled = scaler.transform(patient_data)

        # Predict
        pred_log = log_model.predict(patient_scaled)[0]
        pred_tree = tree_model.predict(patient_data)[0]

        # Show results
        result_df = pd.DataFrame({
            'Model':['Logistic Regression','Decision Tree'],
            'Prediction':[pred_log, pred_tree],
            'Outcome':['Diabetes (1)' if pred_log==1 else 'No Diabetes (0)',
                       'Diabetes (1)' if pred_tree==1 else 'No Diabetes (0)']
        })
        display(result_df)

In [8]:
# 4️⃣ Link Button
button.on_click(on_button_clicked)

In [9]:
# 5️⃣ Display Widgets
display(Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, DiabetesPedigreeFunction, Age, button, output)

FloatText(value=0.0, description='Pregnancies')

FloatText(value=120.0, description='Glucose')

FloatText(value=70.0, description='BloodPressure')

FloatText(value=20.0, description='SkinThickness')

FloatText(value=79.0, description='Insulin')

FloatText(value=25.0, description='BMI')

FloatText(value=0.5, description='DPF')

FloatText(value=30.0, description='Age')

Button(description='Predict', style=ButtonStyle())

Output()